In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import seaborn as sb

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [1]:
totalData = pd.read_csv('totalData.csv')
totalData.drop(['Unnamed: 0'], inplace=True, axis=1)
totalData.head()

NameError: name 'pd' is not defined

In [ ]:
# import preprocess_kgptalkie as ps
# totalData['title'] = totalData['title'].apply(lambda x: ps.remove_special_chars(x))
totalData = totalData[totalData['title'].notnull()]
# totalData.head()
y = totalData['class'].values
x = [d.split() for d in totalData['title'].tolist()]

# print(x.)
# print(y)

In [ ]:
## converting the words to vectors
wordToVec = gensim.models.Word2Vec(sentences=x, size=100, window=10, min_count=1)
len(wordToVec.wv.vocab)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)

In [ ]:
plt.hist([len(y) for y in x], bins=700)

In [ ]:
array = np.array([len(y) for y in x])
maximumLength = 1000
x = pad_sequences(x, maxlen=maximumLength)

In [ ]:
sizeOfVocab = len(tokenizer.word_index) + 1
totalVocab = tokenizer.word_index


In [ ]:
def calculateMatrixWeight(model):
    matrixWeight = np.zeros((sizeOfVocab, 100))
    for word, index in totalVocab.items():
        matrixWeight[index]=model.wv[word]
    return matrixWeight

In [ ]:
vectorsToEmbed = calculateMatrixWeight(wordToVec)
vectorsToEmbed.shape

In [ ]:
totalModel = Sequential()
totalModel.add(Embedding(sizeOfVocab, output_dim=100, weights=[vectorsToEmbed], input_length=maximumLength, trainable=False))
totalModel.add(LSTM(units=128))
totalModel.add(Dense(1, activation='sigmoid'))
totalModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
%pip install keras


In [ ]:
totalModel.summary()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y)
totalModel.fit(x_train,y_train, validation_split=0.3, epochs=6)
# print(y.shape)
# print(x.shape)

In [ ]:
y_pred_train = (totalModel.predict(x_train) >= 0.5).astype(int)
y_pred_test = (totalModel.predict(x_test) >= 0.5).astype(int)
print('The accuracy for test: {}'.format(accuracy_score(y_test, y_pred_test)))

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(20, 8))

from sklearn.metrics import confusion_matrix
train = sb.heatmap(confusion_matrix(y_train, y_pred_train), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax=axes[0])
train.set( xlabel = "Train Data")


test = sb.heatmap(confusion_matrix(y_test, y_pred_test), 
           annot = True, fmt=".0f", annot_kws={"size": 18}, ax=axes[1])
test.set(xlabel="Test Data")

In [ ]:
x = ['hello there how are you']
x = tokenizer.texts_to_sequences(x)
x = pad_sequences(x, maxlen=maximumLength)
(totalModel.predict(x)>=0.5).astype(int)